In [8]:
import pandas as pd
import numpy as np
import os 



In [40]:
file_path = "../Data/Amazon Sale Report.csv"

df = pd.read_csv(file_path, low_memory=False)

#* Cleaning column names
df.columns = df.columns.str.replace(' ', '_')

#* Filling missing values for specific columns
df['Courier_Status'] = df['Courier_Status'].fillna('unknown')
df['promotion-ids'] = df['promotion-ids'].fillna('no promotion')
df.drop(columns=['index','Unnamed:_22'], inplace=True)

#print(df.count())
print(df.columns)


Index(['Order_ID', 'Date', 'Status', 'Fulfilment', 'Sales_Channel_',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier_Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by'],
      dtype='object')


In [61]:
#* customer dimension table

dim_customer = df[['ship-city', 'ship-state', 'ship-postal-code', 'ship-country']].drop_duplicates().dropna()
dim_customer.reset_index(drop=True, inplace=True)
dim_customer.insert(0, 'customer_id', dim_customer.index + 1)

print(dim_customer.head())


   customer_id    ship-city   ship-state  ship-postal-code ship-country
0            1       MUMBAI  MAHARASHTRA          400081.0           IN
1            2    BENGALURU    KARNATAKA          560085.0           IN
2            3  NAVI MUMBAI  MAHARASHTRA          410210.0           IN
3            4   PUDUCHERRY   PUDUCHERRY          605008.0           IN
4            5      CHENNAI   TAMIL NADU          600073.0           IN


In [ ]:

#* creating dim table for the date 
df['Date'] = pd.to_datetime(df['Date'], format='%m-%d-%y', errors='coerce')
dim_date = df[['Date']].drop_duplicates().dropna().rename(columns={'Date': 'order_date'})
dim_date['year'] = dim_date['order_date'].dt.year
dim_date['month'] = dim_date['order_date'].dt.month
dim_date['day'] = dim_date['order_date'].dt.day
dim_date.reset_index(drop=True, inplace=True)
dim_date.insert(0, 'date_id', dim_date.index + 1)

print(dim_date.columns)

Index(['date_id', 'order_date', 'year', 'month', 'day'], dtype='object')


In [37]:
dim_product = df[['SKU', 'Category', 'Style', 'Size']].drop_duplicates().dropna()
dim_product.reset_index(drop=True, inplace=True)
dim_product.insert(0, 'product_id', dim_product.index + 1)
print(dim_product.columns)

Index(['product_id', 'SKU', 'Category', 'Style', 'Size'], dtype='object')


In [ ]:
dim_fulfillment = df[['Fulfilment', 'Sales_Channel_', 'ship-service-level']].drop_duplicates().dropna()
dim_fulfillment.reset_index(drop=True, inplace=True)
dim_fulfillment.insert(0, 'fulfillment_id', dim_fulfillment.index + 1)
#print(dim_fulfillment)

   fulfillment_id Fulfilment Sales_Channel_ ship-service-level
0               1   Merchant      Amazon.in           Standard
1               2     Amazon      Amazon.in          Expedited
2               3     Amazon      Amazon.in           Standard
3               4     Amazon     Non-Amazon           Standard


In [45]:
dim_promotion = df[['promotion-ids']].drop_duplicates().dropna()
dim_promotion.reset_index(drop=True, inplace=True)
dim_promotion.insert(0, 'promotion_id', dim_promotion.index + 1)
print(dim_promotion)

      promotion_id                                      promotion-ids
0                1                                       no promotion
1                2  Amazon PLCC Free-Financing Universal Merchant ...
2                3       IN Core Free Shipping 2015/04/08 23-48-5-108
3                4  Amazon PLCC Free-Financing Universal Merchant ...
4                5  Amazon PLCC Free-Financing Universal Merchant ...
...            ...                                                ...
5783          5784  Amazon PLCC Free-Financing Universal Merchant ...
5784          5785  Amazon PLCC Free-Financing Universal Merchant ...
5785          5786  Amazon PLCC Free-Financing Universal Merchant ...
5786          5787  Amazon PLCC Free-Financing Universal Merchant ...
5787          5788  Amazon PLCC Free-Financing Universal Merchant ...

[5788 rows x 2 columns]


In [48]:
dim_status = df[['Status']].drop_duplicates().dropna().rename(columns={'Status': 'status_name'})
dim_status.reset_index(drop=True, inplace=True)
dim_status.insert(0, 'status_id', dim_status.index + 1)
print(dim_status)


    status_id                    status_name
0           1                      Cancelled
1           2   Shipped - Delivered to Buyer
2           3                        Shipped
3           4   Shipped - Returned to Seller
4           5    Shipped - Rejected by Buyer
5           6      Shipped - Lost in Transit
6           7     Shipped - Out for Delivery
7           8  Shipped - Returning to Seller
8           9            Shipped - Picked Up
9          10                        Pending
10         11  Pending - Waiting for Pick Up
11         12              Shipped - Damaged
12         13                       Shipping


In [50]:
dim_courier_status = df[['Courier_Status']].drop_duplicates().dropna().rename(columns={'Courier_Status': 'courier_status_name'})
dim_courier_status.reset_index(drop=True, inplace=True)
dim_courier_status.insert(0, 'courier_status_id', dim_courier_status.index + 1)
print(dim_courier_status)

   courier_status_id courier_status_name
0                  1             unknown
1                  2             Shipped
2                  3           Cancelled
3                  4           Unshipped


In [ ]:
 
fact_sales = df[['Order_ID', 'Date', 'ship-city', 'ship-state', 'ship-postal-code', 'ship-country', 'SKU', 'Category', 'Style', 'Fulfilment', 'Sales_Channel_', 'ship-service-level', 'promotion-ids', 'Status', 'Courier_Status', 'Qty', 'Amount', 'currency', 'B2B', 'fulfilled-by']].dropna()

 
df['Date'] = pd.to_datetime(df['Date'], format='%m-%d-%y', errors='coerce')
fact_sales['Date'] = pd.to_datetime(fact_sales['Date'], format='%m-%d-%y', errors='coerce')

# Merge fact_sales with dimension tables
fact_sales = fact_sales.merge(dim_customer, on=['ship-city', 'ship-state', 'ship-postal-code', 'ship-country'], how='left')
fact_sales = fact_sales.merge(dim_product, on=['SKU', 'Category', 'Style'], how='left')
fact_sales = fact_sales.merge(dim_fulfillment, on=['Fulfilment', 'Sales_Channel_', 'ship-service-level'], how='left')
fact_sales = fact_sales.merge(dim_promotion, on=['promotion-ids'], how='left')
fact_sales = fact_sales.merge(dim_status, left_on='Status', right_on='status_name', how='left').drop(columns=['status_name'])
fact_sales = fact_sales.merge(dim_courier_status, left_on='Courier_Status', right_on='courier_status_name', how='left').drop(columns=['courier_status_name'])

 
fact_sales = fact_sales.merge(dim_date, left_on='Date', right_on='order_date', how='left')

 
fact_sales = fact_sales.rename(columns={'Order_ID': 'order_id', 'Qty': 'quantity', 'Amount': 'amount'})

 
fact_sales = fact_sales[['order_id', 'date_id', 'customer_id', 'product_id', 'fulfillment_id', 'promotion_id', 'status_id', 'courier_status_id', 'quantity', 'amount', 'currency', 'B2B', 'fulfilled-by']]

 
fact_sales.reset_index(drop=True, inplace=True)


print(fact_sales.columns)

Index(['order_id', 'date_id', 'customer_id', 'product_id', 'fulfillment_id',
       'promotion_id', 'status_id', 'courier_status_id', 'quantity', 'amount',
       'currency', 'B2B', 'fulfilled-by'],
      dtype='object')


In [78]:
#* saving the dimension table to a csv file
dim_customer.to_csv('../Data/dim_customer.csv', index=False)
dim_date.to_csv('../Data/dim_date.csv', index=False)
dim_product.to_csv('../Data/dim_product.csv', index=False)
dim_fulfillment.to_csv('../Data/dim_fulfillment.csv', index=False)
dim_promotion.to_csv('../Data/dim_promotion.csv', index=False)
dim_status.to_csv('../Data/dim_status.csv', index=False)
dim_courier_status.to_csv('../Data/dim_courier_status.csv', index=False)
fact_sales.to_csv("../Data/fact_sales.csv", index=False)
